In [ ]:
! pip install datasets
! pip install transformers==4.45.2 sentence-transformers==3.1.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoModelForCausalLM, AutoTokenizer,TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import torch
# 加载 Llama2-7B
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"  # 适用于 GPT 类模型
)
model = get_peft_model(model, lora_config)  # 应用 LoRA
tokenizer = AutoTokenizer.from_pretrained(model_name)




In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
# 配置 LoRA 适配层
# 应用 LoRA
model = get_peft_model(model, lora_config)

In [ ]:
device = model.device

In [ ]:
! ls

In [ ]:


from sklearn.metrics import accuracy_score, classification_report
# 3️⃣ 处理数据
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    inputs["labels"] = [float(label) for label in examples["label"]]  # 确保 label 是 float
    return inputs

dataset = load_dataset("json", data_files={"train": "drive/MyDrive/Colab/llm_demo/train.json", "test": "drive/MyDrive/Colab/llm_demo/train.json"})
tokenized_datasets = dataset.map(preprocess_function, batched=True)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# 4️⃣ 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    logging_dir="./logs",
    learning_rate=2e-5
)



# 6️⃣ 训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: haoyuh (haoyuh-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.580140,0.500000
2,No log,1.471590,0.500000
3,No log,1.384634,0.500000
4,No log,1.331336,0.500000
5,No log,1.308993,0.500000


TrainOutput(global_step=65, training_loss=1.4566213754507211, metrics={'train_runtime': 542.2231, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.12, 'total_flos': 16380283944960.0, 'train_loss': 1.4566213754507211, 'epoch': 5.0})

In [ ]:
import torch
import torch.nn.functional as F

def predict(text):
    # 确保模型在正确的设备上
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 预处理输入文本
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256).to(device)

    # 计算 logits（模型原始输出）
    with torch.no_grad():
        logits = model(**inputs).logits  # logits 形状: (1, 1)
        prob = torch.sigmoid(logits)  # 计算概率（0~1）
    probs = torch.softmax(logits, dim=-1)  # 使用 softmax 获取概率
    class_1_probs = probs[:, 1].numpy()    # 获取类别 1 的概率

    return class_1_probs  # 返回单个数值




In [ ]:
# **测试预测**
text1 = "用户过去一年内信用评分保持稳定，无不良记录。"
text2 = "用户近期信用卡账单连续三个月未全额还款，负债率较高。"
text3 = "用户最近六个月内按时还款，无逾期记录。"

print(f"Text1 逾期概率: {predict(text1)[0]:.4f}")
print(f"Text2 逾期概率: {predict(text2)[0]:.4f}")
print(f"Text3 逾期概率: {predict(text3)[0]:.4f}")

Text1 逾期概率: 0.8737
Text2 逾期概率: 0.9268
Text3 逾期概率: 0.9017


In [ ]:
prob1[0]

NameError: name 'prob1' is not defined